In [6]:
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
plt.rcParams["font.size"] = 16

In [8]:
def process_file(file_path):
    data = pd.read_csv(file_path, sep="\t")

    data["dfs_dT"] = np.abs(np.gradient(data["fs"], data["T"]))
    fs_dot5_index = np.abs(data["fs"] - 0.50).idxmin()
    diamond_index = data["f_tot(@Diamond)"].first_valid_index()

    return {
        "T": data["T"],
        "fs": data["fs"],
        "w": float(file_path.stem[0]),
        "delta_T_SL": data["T"].iloc[0] - data["T"].iloc[-1],
        "dfs_dT": data["dfs_dT"].iloc[fs_dot5_index],
        "alpha_Al": data["f_tot(@Fcc)"].iloc[diamond_index - 1],
        "Mg2Si": data["f_tot(@Mg2Si)"].iloc[-1],
    }

In [ ]:
plt.figure(figsize=(16, 10))

data_dir = Path("./data/0.5Mg").glob("*.txt")
results = [process_file(file) for file in data_dir]

df = pd.DataFrame(results)

for result in results:
    plt.plot(result["T"], (1 - result["fs"]) * 100, label=f"Al-0.5Mg-{result['w']}")

plt.xlabel("Temperature ($^\\circ C$)")
plt.ylabel("Liquid Fraction (%)")
plt.legend()

# plt.savefig("image/fig-fix_Mg-1.png", dpi=200)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 10))
fig.subplots_adjust(right=0.65)

twin1 = ax.twinx()
twin2 = ax.twinx()
twin3 = ax.twinx()

twin2.spines.right.set_position(("axes", 1.2))
twin3.spines.right.set_position(("axes", 1.4))

(p1,) = ax.plot(
    df["w"],
    df["delta_T_SL"],
    "C0",
    marker="s",
    label="Solidification interval, $\\Delta T^{S/L}$",
)
(p2,) = twin1.plot(
    df["w"],
    df["dfs_dT"],
    "C1",
    marker="^",
    label="$(dfs/dT)_{f=0.5}$",
)
(p3,) = twin2.plot(
    df["w"],
    df["alpha_Al"] * 100,
    "C2",
    marker="o",
    label="$\\alpha-Al$ (%)",
)
(p4,) = twin3.plot(
    df["w"],
    df["Mg2Si"] * 100,
    "C3",
    marker="d",
    label="$Mg_{2}Si$ (%)",
)

ax.set(xlabel="w(Si) (%)", ylabel="$\\Delta T^{S/L} (^{\\circ}C$)")
twin1.set(ylabel="$(dfs/dT)_{f=0.5} (K^{-1})$")
twin2.set(ylabel="$\\alpha-Al$ (%)")
twin3.set(ylabel="$Mg_{2}Si$ (%)")

ax.yaxis.label.set_color(p1.get_color())
twin1.yaxis.label.set_color(p2.get_color())
twin2.yaxis.label.set_color(p3.get_color())
twin3.yaxis.label.set_color(p4.get_color())

ax.tick_params(axis="y", colors=p1.get_color())
twin1.tick_params(axis="y", colors=p2.get_color())
twin2.tick_params(axis="y", colors=p3.get_color())
twin3.tick_params(axis="y", colors=p4.get_color())

ax.legend(loc="center left")
twin1.legend(loc="center right", bbox_to_anchor=(1, 0.6))
twin2.legend(loc="center right", bbox_to_anchor=(1, 0.5))
twin3.legend(loc="center right", bbox_to_anchor=(1, 0.4))

# plt.savefig("image/fig-fix_Mg-2.png", dpi=200)
plt.show()